In [1]:
import os
import time
import pandas as pd
import numpy as np
import pathlib
from io import StringIO
from Bio import SeqIO
from Bio.Cluster import distancematrix
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import subprocess

Make sure blast is on path

in .bashrc add

export PATH="$PATH:~/somefolder/ncbi-blast-2.11.0+/bin"

before running this notebook run:

export BLASTDB=/somepath/database_directory

eg

export BLASTDB=/mnt/255GB_ssd/Data/BLAST/blastdb

concatenated all reads to one fastq: for i in `ls *.fastq`; do cat $i >> ICU10G_S4_L001_R1R2.fastq; echo "Completed"; done

In [2]:
PRJ='PRJNA686836'

DATA_PATH=f'/mnt/1TB_0/Data/Assembly/{PRJ}/'


DB='nt'
#DB='gsa_bsl_nt_db'

METAXA_FILE=f'metaxa2.extraction.fasta'


OUT_FILE_UID='metaxa2_rRNA'
SRA_PATH_EXTENSION='metaxa2/'

#percentage identity 95%
#PER_ID = 95
PER_ID = 80
#Expect value (E) for saving hits
#E_VAL=0.001
E_VAL=0.05
NUM_THREADS=4
MAX_TGT_SEQS=100
#Maximum number of HSPs (alignments) to keep for any single query-subject pair.
#If this option is not set (None), BLAST shows all HSPs meeting the expect value criteria.
MAX_HSPS=1
RUN_CODE=f'seq{MAX_TGT_SEQS}_hsps{MAX_HSPS}'

#for random

NUM_READS=1000
RUN_NUM=1

In [3]:
def get_reads(sra_file):
    readlist=[]
    idlist=[]
    for record in SeqIO.parse(sra_file, "fasta"):
        idlist.append(record.id)
        readlist.append(record.seq)
    return idlist, readlist

In [4]:
def get_rand_reds(readlist, run_file):
    #random reads
    rand_vals=list(np.random.choice(len(readlist), NUM_READS))
    df = pd.DataFrame(rand_vals, columns=["random_vals"])
    df.to_csv(run_file, index=False)
    return rand_vals

In [5]:
def get_contigs(contigs_file):
    contigs={}
    filec = open(contigs_file, 'r')
    lines = filec.readlines()
    prev_line = None
    for line in lines:
        line = line.strip('\n')
        contig_id=line.split(' ')[0]
        #for metaxa fasta results split on pipe
        contig_id=line.split('|')[0]
        if not line.startswith('>'):
            if prev_line==None:
                prev_line=contig_id
            contigs[prev_line]=line
        prev_line=contig_id
    return contigs
    

### Local Blast

In [6]:
#local BLAST
#see https://www.biostars.org/p/332113/
def search(idx, query_string, database_path, alfile, hspfile):
    start_time=time.time()

    per_id = PER_ID
    e_val=E_VAL
    max_tgt_seqs=MAX_TGT_SEQS
    max_hsps=MAX_HSPS
    blastn_cline = NcbiblastnCommandline(
                                         db=database_path, 
                                         evalue=e_val,
                                         outfmt=5, 
                                         perc_identity=per_id,
                                         max_target_seqs=max_tgt_seqs, 
                                         max_hsps=max_hsps,
                                         num_threads=NUM_THREADS
                                        )
    out, err = blastn_cline(stdin=query_string)
    io_result = StringIO(out)
    blast_records = list(NCBIXML.parse(io_result))
    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            title = alignment.title
            query_length = blast_record.query_letters
            alfile.write(f"id: {idx}, title: {title}, accession: {alignment.accession}\n")
            for hsp in alignment.hsps:
                    hspfile.write(f"id: {idx}, title: {title}, accession: {alignment.accession}, \
                    hit_id: {alignment.hit_id}, length: {alignment.length}, query_length {blast_record.query_letters}, \
                    score: {hsp.score}, expect: {hsp.expect}, align_length: {hsp.align_length}, bits: {hsp.bits}, \
                    query: {hsp.query}, sbjct: {hsp.sbjct}, query_start: {hsp.query_start}, query_end: {hsp.query_end}, sbjct_start: {hsp.sbjct_start}, sbjct_end: {hsp.sbjct_end}\n")
    return 

In [7]:
def run_randomised_subset(sra):
    sra_path=DATA_PATH+sra+'/'+SRA_PATH_EXTENSION
    blast_path=sra_path+'blast/'
    pathlib.Path(blast_path).mkdir(exist_ok=True)
    contigs_file=sra_path+METAXA_FILE
    
    idlist, readlist = get_reads(contigs_file)
    
    run_file=f'{blast_path}{sra}_{OUT_FILE_UID}_blast_{DB}_{RUN_CODE}_PCT{PER_ID}_E{E_VAL}_{NUM_READS}_indexes_run_{RUN_NUM}.csv'
    result_file=f'{blast_path}{sra}_{OUT_FILE_UID}_blast_{DB}_{RUN_CODE}_PCT{PER_ID}_E{E_VAL}_{NUM_READS}_run_{RUN_NUM}.csv'
    hsp_file=f'{blast_path}{sra}_{OUT_FILE_UID}_blast_{DB}_{RUN_CODE}_PCT{PER_ID}_E{E_VAL}_hsps_{NUM_READS}_run_{RUN_NUM}.txt'
    hit_data=[]
    
    rand_vals=get_rand_reds(readlist, run_file)
    #open file first and write as we go
    resultf= open(result_file,"a")
    hspf= open(hsp_file,"a")
    i=0
    for ri in rand_vals:
        if (i % 10000==0) and i>0:
            print(f'Blasting contig: {i} index: {k} elapsed: {time.time()-start_time}')
        if (i % 100==0):
            resultf.flush()
            hspf.flush()
        #idx, query_string, database_path, outfile
        hit_data.append(search(ri, str(readlist[ri]), DB, resultf, hspf))
        i+=1
    resultf.close()
    hspf.close()
    hspf.close()
    

In [8]:
def run_blast(sra):
    loop_time=time.time()
    sra_path=DATA_PATH+sra+'/'+SRA_PATH_EXTENSION
    blast_path=sra_path+'blast/'
    pathlib.Path(blast_path).mkdir(exist_ok=True)
    contigs_file=sra_path+METAXA_FILE
    contigs = get_contigs(contigs_file)
    print(f'Running BLAST on {sra} with {len(contigs)} contigs')
    
    result_file=f'{blast_path}{sra}_{OUT_FILE_UID}_blast_{DB}_{RUN_CODE}_PCT{PER_ID}_E{E_VAL}.csv'
    hsp_file=f'{blast_path}{sra}_{OUT_FILE_UID}_blast_{DB}_{RUN_CODE}_PCT{PER_ID}_E{E_VAL}_hsps.txt'
    hit_data=[]
    #open file first and write as we go
    resultf= open(result_file,"a")
    hspf= open(hsp_file,"a")
    i=0
    for k,v in contigs.items():
        if (i % 10000==0) and i>0:
            print(f'Blasting contig: {i} index: {k} elapsed: {time.time()-start_time}')
        if (i % 100==0):
            resultf.flush()
            hspf.flush()
        #idx, query_string, database_path, outfile
        hit_data.append(search(k, str(v), DB, resultf, hspf))
        i+=1
    resultf.close()
    hspf.close()
    print(f'{sra} completed, in: {time.time()-loop_time}')

In [9]:
#done 'SRR11093266','SRR11093267','SRR11093268','SRR11093269'
#no rRNA: 'SRR11093265',
#SRAs=['SRR11093266','SRR11093267','SRR11093268','SRR11093269']
SRAs=['SRR13285085']

In [10]:
start_time=time.time()
for sra in SRAs:
    run_randomised_subset(sra)
print(f'Total leapsed: {time.time()-start_time}')

Total leapsed: 23841.070493221283
